In [3]:
import pandas as pd
import time
df = pd.read_csv(r"C:\Users\lyons\OneDrive\Desktop\IML-FALL2025---Voter-Bias\original_data\college_football_polls_original.csv")
dff = pd.read_csv(r'C:\Users\lyons\OneDrive\Desktop\IML-FALL2025---Voter-Bias\original_data\college_basketball_polls_original.csv')
#run everything after cooldown passes -> did it too many times while testing whoops 

In [ ]:
#inital test
season_2014 = df[df["Season"] == '2014']
long2014 = season_2014.melt(
    id_vars=['Season', 'Week', 'Pollster'],
    value_vars=[str(i) for i in range(1, 26)],
    var_name='Rank',
    value_name='Team'
).dropna(subset=['Team'])
rank_points = {str(i): 26 - i for i in range(1, 26)}
long2014['Points'] = long2014['Rank'].map(rank_points)
df2014 = long2014.groupby(['Season', 'Week', 'Team'], as_index=False)['Points'].sum()
df2014 = df2014.rename(columns={'Points': 'Total_Points'})
url = 'https://www.sports-reference.com/cfb/years/2014-standings.html'
table = pd.read_html(url)
team_df = table[0]
team_df = team_df.rename(columns={
    'Unnamed: 1_level_0': 'School',
    'Unnamed: 2_level_0': 'Conf'
})
team_df = team_df[['School', 'Conf']]
team_df.columns = team_df.columns.get_level_values(0)
team_df = team_df.dropna()
team_df = team_df[team_df.School != 'School']
team_df.School = team_df.School.str.lower().str.replace(" ", "-").str.replace("&", "and").str.replace("(", "").str.replace(")", "").str.replace(".", "").str.replace("'", "")   

school_to_mascot = df2014['Team'].unique().tolist()
schools = team_df.School.unique().tolist()
map_school_to_mascot = {}
for school in schools:
    matches = [t for t in school_to_mascot if t.startswith(school)]
    if matches:
        map_school_to_mascot[school] = matches[0]
    else: 
        continue

team_df['updated school'] = team_df['School'].map(map_school_to_mascot)
team_df = team_df.dropna()
conf_map_2014 = dict(zip(team_df['updated school'], team_df['Conf']))
df2014['Conference'] = df2014['Team'].map(conf_map_2014)
df2014 = df2014 = df2014[['Season', 'Week', 'Team', 'Conference', 'Total_Points']]

In [1]:
#cfb
def process_poll_data_cfb(df, year):
    season_df = df[df["Season"] == str(year)]
    long_df = season_df.melt(
        id_vars=['Season', 'Week', 'Pollster'],
        value_vars=[str(i) for i in range(1, 26)],
        var_name='Rank',
        value_name='Team'
    ).dropna(subset=['Team'])
    rank_points = {str(i): 26 - i for i in range(1, 26)}
    long_df['Points'] = long_df['Rank'].map(rank_points)
    poll_df = long_df.groupby(['Season', 'Week', 'Team'], as_index=False)['Points'].sum()
    poll_df = poll_df.rename(columns={'Points': 'Total_Points'})
    url = f'https://www.sports-reference.com/cfb/years/{year}-standings.html' # <- link is different for cbb
    table = pd.read_html(url)[0]
    team_df = table.rename(columns={'Unnamed: 1_level_0': 'School', 'Unnamed: 2_level_0': 'Conf'})
    team_df = team_df[['School', 'Conf']]
    team_df.columns = team_df.columns.get_level_values(0)
    team_df = team_df.dropna()
    team_df = team_df[team_df.School != 'School']
    team_df.School = (
        team_df.School.str.lower()
        .str.replace(" ", "-", regex=False)
        .str.replace("&", "and", regex=False)
        .str.replace("(", "", regex=False)
        .str.replace(")", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace("'", "", regex=False)
    )
    school_to_mascot = poll_df['Team'].unique().tolist()
    schools = team_df.School.unique().tolist()
    map_school_to_mascot = {}
    for school in schools:
        matches = [t for t in school_to_mascot if t.startswith(school)]
        if matches:
            map_school_to_mascot[school] = matches[0]
    team_df['updated_school'] = team_df['School'].map(map_school_to_mascot)
    team_df = team_df.dropna()
    conf_map = dict(zip(team_df['updated_school'], team_df['Conf']))
    poll_df['Conference'] = poll_df['Team'].map(conf_map)
    poll_df = poll_df[['Season', 'Week', 'Team', 'Conference', 'Total_Points']]
    return poll_df

In [ ]:
#cfb
def process_all_years_cfb(df):
    all_dfs = []
    for year in sorted(df.Season.unique()):
        try:
            season_df = process_poll_data_cfb(df, year)
            all_dfs.append(season_df)
            print(f"Processed {year}")
            time.sleep(5)#
        except Exception as e:
            print(f"Skipped {year} due to error: {e}")
    all_years_df = pd.concat(all_dfs, ignore_index = True)
    return all_years_df

In [ ]:
#cfb
#process_all_years(df).to_csv(r'C:\Users\lyons\OneDrive\Desktop\IML-FALL2025---Voter-Bias\output_data\cfb')

In [ ]:
#cbb
def process_poll_data_cbb(df, year):
    season_df = df[df["Season"] == str(year)]
    long_df = season_df.melt(
        id_vars=['Season', 'Week', 'Pollster'],
        value_vars=[str(i) for i in range(1, 26)],
        var_name='Rank',
        value_name='Team'
    ).dropna(subset=['Team'])
    rank_points = {str(i): 26 - i for i in range(1, 26)}
    long_df['Points'] = long_df['Rank'].map(rank_points)
    poll_df = long_df.groupby(['Season', 'Week', 'Team'], as_index=False)['Points'].sum()
    poll_df = poll_df.rename(columns={'Points': 'Total_Points'})
    url = f'https://www.sports-reference.com/cbb/seasons/men/2025-standings.html' # <- link for college mens basketball
    table = pd.read_html(url)[0]
    team_df = table.rename(columns={'Unnamed: 1_level_0': 'School', 'Unnamed: 2_level_0': 'Conf'})
    team_df = team_df[['School', 'Conf']]
    team_df.columns = team_df.columns.get_level_values(0)
    team_df = team_df.dropna()
    team_df = team_df[team_df.School != 'School']
    team_df.School = (
        team_df.School.str.lower()
        .str.replace(" ", "-", regex=False)
        .str.replace("&", "and", regex=False)
        .str.replace("(", "", regex=False)
        .str.replace(")", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace("'", "", regex=False)
    )
    school_to_mascot = poll_df['Team'].unique().tolist()
    schools = team_df.School.unique().tolist()
    map_school_to_mascot = {}
    for school in schools:
        matches = [t for t in school_to_mascot if t.startswith(school)]
        if matches:
            map_school_to_mascot[school] = matches[0]
    team_df['updated_school'] = team_df['School'].map(map_school_to_mascot)
    team_df = team_df.dropna()
    conf_map = dict(zip(team_df['updated_school'], team_df['Conf']))
    poll_df['Conference'] = poll_df['Team'].map(conf_map)
    poll_df = poll_df[['Season', 'Week', 'Team', 'Conference', 'Total_Points']]
    return poll_df

In [ ]:
#cbb
def process_all_years_cbb(df):
    all_dfs = []
    for year in sorted(df.Season.unique()):
        try:
            season_df = process_poll_data_cbb(df, year)
            all_dfs.append(season_df)
            print(f"Processed {year}")
            time.sleep(5)#
        except Exception as e:
            print(f"Skipped {year} due to error: {e}")
    all_years_df = pd.concat(all_dfs, ignore_index = True)
    return all_years_df

In [ ]:
#cbb
#process_all_years(dff).to_csv(r'C:\Users\lyons\OneDrive\Desktop\IML-FALL2025---Voter-Bias\output_data\cbb')